In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# 🔑 Sett inn din API-nøkkel (Client ID fra Frost API)
CLIENT_ID = "2a430097-ac9b-44e4-995e-a02afb9795b2"  # Erstatt dette med din riktige nøkkel
API_URL = "https://frost.met.no/observations/v0.jsonld"

# 🌍 Velg værstasjon i Trondheim (SN68860 = Trondheim - Voll)
SOURCE = "SN68860"

# 🌡️ Hvilke værdata vi ønsker å hente
ELEMENTS = "air_temperature,cloud_area_fraction,air_pressure_at_sea_level,precipitation_amount"

# 📅 Tidsperiode (siste 30 dager)
slutt_dato = datetime.utcnow().date()
start_dato = slutt_dato - timedelta(days=30)
tidspunkt = f"{start_dato}/{slutt_dato}"

# 🛠️ Bygg API-forespørselen
params = {
    "sources": SOURCE,
    "elements": ELEMENTS,
    "referencetime": tidspunkt
}

# 🔄 Send forespørselen til Frost API
response = requests.get(API_URL, params=params, auth=(CLIENT_ID, ""))

# 🚀 Sjekk om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()["data"]
    
    # 📊 Lagring av værdata (med alle parametere)
    værdata = {}
    for entry in data:
        dato = entry["referenceTime"].split("T")[0]  # Hent dato
        element = entry["observations"][0]["elementId"]  # Hent datatype
        verdi = entry["observations"][0]["value"]  # Hent målt verdi
        
        # Opprett en rad for hver dato hvis den ikke eksisterer
        if dato not in værdata:
            værdata[dato] = {
                "Dato": dato,
                "Temperatur (°C)": None,
                "Skydekke (%)": None,
                "Lufttrykk (hPa)": None,
                "Nedbør (mm)": None
            }

        # Legg til målingene i riktig kolonne
        if element == "air_temperature":
            værdata[dato]["Temperatur (°C)"] = verdi
        elif element == "cloud_area_fraction":
            værdata[dato]["Skydekke (%)"] = verdi
        elif element == "air_pressure_at_sea_level":
            værdata[dato]["Lufttrykk (hPa)"] = verdi
        elif element == "precipitation_amount":
            værdata[dato]["Nedbør (mm)"] = verdi

    # Konverter til Pandas DataFrame
    df = pd.DataFrame(list(værdata.values()))

    # 🚀 Fyll inn eventuelle manglende verdier med "0"
    df.fillna(0, inplace=True)

    # 📁 Lagre som CSV-fil
    filnavn = "frost_trondheim_30dager3.csv"
    df.to_csv(filnavn, index=False, encoding="utf-8")

    print(f"✅ Værdata lagret i {filnavn}")

else:
    print(f"❌ Feil ved henting av data: {response.status_code} - {response.text}")


✅ Værdata lagret i frost_trondheim_30dager3.csv
